In [99]:
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [4]:
uri = 'mongodb://172.20.23.138:27017'
client = MongoClient(uri)
db = client['ptt']
collect = db['intltrade']

In [61]:

# db = client['findbiz']
# collect = db['companies']

# query1 = collect.find({"$and": [{'登記機關': {'$eq': '臺中市政府'}},
#           {"實收資本額(元)": {"$gt": 10000000}}]})
# query1 = collect.find({"資本總額(元)": {"$gt": 10}})
query1 = collect.find({})

In [109]:
import jieba
import jieba.analyse
import re
stopwordPath = '/usr/local/lib/python3.5/dist-packages/jieba/chinese_stopword.txt'
stop_word = [line.rstrip() for line in open(stopwordPath)]
jieba.load_userdict('/usr/local/lib/python3.5/dist-packages/jieba/user.dict')

In [110]:
query1 = collect.find({})

In [111]:
corpus = []
titleList = []
while 1:
    try:
        tmp = query1.next()
    except:
        break
    if not tmp['length']: continue
    try:
        timeStamp = int(re.findall('[0-9]{4}',tmp['timeStamp'])[0])
    except:
        continue
    if timeStamp not in [2017, 2018]: continue

    content = ''
    content += tmp['title'] * 2 # Adjust the weight of term in title
    for push in tmp['mainContent']:
        if push.get('type') == 'main':
            content += push.get('content')
        else:
            content += push['content']['pushContent']

    content = content.replace('\n', '').replace('《', '').replace('》', '')
    seg_list = jieba.cut(content)#, cut_all=True)
    result = ' '.join([re.sub(r"[^\u4E00-\u9FFF\u3000-\u303Fa-zA-Z]", '', i) for i in seg_list if i not in stop_word])
    corpus.append(result)
    titleList.append(tmp['title'])

In [112]:
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(corpus)
weight = tfs.toarray()
words = tfidf.get_feature_names()

In [113]:
n = 5 # 前五位
for (title, w) in zip(titleList, weight):
    print('{}:'.format(title))
    # 排序
    loc = np.argsort(-w)
    for i in range(n):
        print('-{}: {} {}'.format(str(i + 1), words[loc[i]], w[loc[i]]))
    print('\n')

[船務] 請問C&F 的出口報單問題:
-1: usd 0.31527197578737304
-2: 報單 0.3071241115369407
-3: 會計 0.2968897179375754
-4: 螺帽 0.27425283034488085
-5: ntd 0.2447257669160934


[問題] 對中國貿易關稅了解的大大:
-1: 關稅 0.3386941940765985
-2: 中國 0.2550725648600413
-3: 大陸 0.2461958619627747
-4: about 0.2419551910321779
-5: it 0.21590540965509084


[問題] 散貨材積的測量問題:
-1: cbm 0.548634177021511
-2: 測量 0.29375594992936305
-3: 散貨材積 0.20899995039826444
-4: 每次 0.20650687557998604
-5: 沒變 0.19583729995290874


[問題] 推薦的出口船務書籍？:
-1: 船務 0.3920611552551723
-2: 書籍 0.2785727303430363
-3: 中學 0.2610240134392343
-4: thatsgreat 0.1857120030238503
-5: 助理 0.17481393920399346


[船務] 請教Lc審單:
-1: 信用狀 0.5204030194419917
-2: 審單 0.2629225430463762
-3: 報關行 0.21873977353548363
-4: 委外 0.21134646058805515
-5: 文件 0.1401667245054973


[問題] 請推薦彰化的報關行:
-1: 彰化 0.6328224655188941
-2: 報關行 0.27959244312236203
-3: nxt 0.22511861280794426
-4: 為主少 0.22511861280794426
-5: 車用 0.22511861280794426


[業務] 尼泊爾到台灣空運:
-1: 尼泊爾到 0.41112120116660184
-2: 估價 0.20556060058330092
-3

-4: 實銷 0.2959276252700276
-5: 實報 0.2959276252700276


[問題] 請問有人有送過金屬中心檢驗嗎:
-1: 檢驗 0.43552953548790213
-2: 中心 0.25204787180847793
-3: 有人 0.2520422297581931
-4: 東西 0.22835655912109132
-5: 過金屬 0.20992896978745637


[問題] 請問有推薦的日本食品報關行嗎:
-1: lovewawa 0.3874418529762704
-2: 中標 0.3630410736033108
-3: 推薦 0.287055740867421
-4: 食品 0.2488729114864268
-5: 報關行 0.24059719649637895


日本進口forwarder:
-1: 日本 0.6618649225771911
-2: 借牌 0.3002859873415453
-3: forwarder 0.17799682668557792
-4: 居所 0.1602344305062624
-5: 再作 0.1602344305062624


[問題] 請問有代付款+報關+物流的公司嗎?:
-1: 公司 0.25637268848030664
-2: 第三方 0.22888965616760026
-3: 代付款 0.19910947011340782
-4: 報關 0.19766238667733385
-5: 採購 0.1779247409745848


Re: [問題] 請問有人有送過金屬中心檢驗嗎:
-1: 檢驗 0.27005250882581394
-2: 中心 0.2604728060131654
-3: 認證 0.21800792275626937
-4: 樣品 0.1938579091016967
-5: 明書 0.16782781569646485


[問題] 北部報關行推薦(食品原料:
-1: 北部 0.4276270965636228
-2: 推薦 0.35505557830597084
-3: 原料 0.3244375341140631
-4: 食品 0.30782772448823953
-5: 報關行 0.2975915983518738

-4: 天津 0.23652353736352646
-5: 尋找 0.19879910123201813


[問題] 日本報關行/forwarder 退稅問題:
-1: 退稅 0.532818264232107
-2: 日本 0.4356541018767429
-3: forwarder 0.24603902479950654
-4: 報關行 0.2445170125118203
-5: 剛剛 0.20945573610327706


Re: [問題] 保固台灣關稅問題:
-1: 課稅 0.34384274412570637
-2: 關稅法 0.267246017673369
-3: 硬碟 0.25041507629350385
-4: 調換 0.20043451325502673
-5: 條規定 0.20043451325502673


[問題] 四角貿易與信用狀問題:
-1: 提單 0.3858913559637026
-2: 日本 0.2792545137352009
-3: 換單 0.26852247910714194
-4: 正本 0.26416451838137445
-5: 信用狀 0.2193464466162307


[問題] 台灣水果出口至關島:
-1: 關島 0.5909218065632181
-2: 水果 0.4394175018381486
-3: 出口 0.2057856988832712
-4: 台灣 0.15173139487933743
-5: 法規 0.14773295783544876


[閒聊]幫求問這份工作的去還是留...:
-1: 朋友 0.3468671240730607
-2: 打單 0.32417632752568926
-3: 工作 0.23328304199448935
-4: 個性 0.2262886877414522
-5: 單櫃 0.1609986947499253


[船務] 面膜出口到美國:
-1: 面膜 0.4760235214228064
-2: 美國 0.3545642112109673
-3: oneplus 0.3386787310680752
-4: 測報 0.1693393655340376
-5: 船務 0.15888732422110427


Re: [問題] 為何

In [146]:
topN = 5
freq = dict()
for i in range(weight.shape[0]):
    for j in range(topN):
        wordTmp = words[np.argsort(-weight[i])[j]]
        if freq.get(wordTmp):
            freq[wordTmp] += 1
        else:
            freq[wordTmp] = 1
len(freq)

1451

In [148]:
{k: v for k, v in freq.items() if v > 1}

{'agent': 2,
 'asus': 2,
 'bank': 2,
 'bb': 2,
 'bl': 2,
 'bs': 3,
 'cbm': 3,
 'charge': 2,
 'cif': 4,
 'cm': 3,
 'com': 2,
 'consignee': 3,
 'dap': 2,
 'dat': 2,
 'date': 3,
 'ddp': 2,
 'dhl': 4,
 'ecfa': 5,
 'ems': 2,
 'en': 3,
 'erp': 2,
 'etd': 2,
 'exporter': 2,
 'fda': 2,
 'fedex': 4,
 'fee': 2,
 'fob': 5,
 'for': 2,
 'form': 3,
 'forwarder': 8,
 'fwd': 6,
 'id': 2,
 'imgur': 3,
 'invoice': 3,
 'isf': 2,
 'iv': 2,
 'kg': 3,
 'lc': 3,
 'line': 3,
 'no': 3,
 'notify': 2,
 'ntd': 2,
 'number': 2,
 'oem': 2,
 'offer': 2,
 'op': 2,
 'pk': 2,
 'registered': 2,
 'rmb': 2,
 'shipper': 3,
 'system': 2,
 'the': 3,
 'tnt': 2,
 'tt': 2,
 'ups': 2,
 'usd': 3,
 '一套': 2,
 '三角': 8,
 '上海': 4,
 '上華': 2,
 '丙方': 2,
 '中國': 9,
 '中心': 2,
 '中文': 2,
 '乙方': 2,
 '乙級': 3,
 '九月': 2,
 '五金': 2,
 '付款': 5,
 '代工': 2,
 '代收': 2,
 '代理': 4,
 '代理商': 4,
 '企劃': 2,
 '伊朗': 7,
 '佣金': 2,
 '來料': 3,
 '供應': 5,
 '侵權': 2,
 '便宜': 2,
 '俄羅斯': 2,
 '信用狀': 12,
 '修改': 4,
 '倉庫': 2,
 '價值': 2,
 '價格': 2,
 '內信': 2,
 '公司': 14,
 '公告': 2,
 '公正

In [151]:
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(freq.items(), key=operator.itemgetter(1), reverse=True)
sorted_x

[('大陸', 23),
 ('貿易', 21),
 ('報關行', 17),
 ('進口', 15),
 ('出口', 14),
 ('公司', 14),
 ('工作', 14),
 ('客人', 13),
 ('信用狀', 12),
 ('國貿', 12),
 ('報單', 12),
 ('廠商', 12),
 ('日本', 11),
 ('稅則', 11),
 ('報關', 11),
 ('美國', 10),
 ('提單', 10),
 ('船務', 10),
 ('香港', 10),
 ('文件', 9),
 ('客戶', 9),
 ('中國', 9),
 ('產品', 8),
 ('工廠', 8),
 ('forwarder', 8),
 ('推薦', 8),
 ('三角', 8),
 ('食品', 7),
 ('清關', 7),
 ('伊朗', 7),
 ('越南', 7),
 ('銀行', 7),
 ('退稅', 7),
 ('德國', 7),
 ('台灣', 7),
 ('空運', 7),
 ('朋友', 7),
 ('貨櫃', 6),
 ('課程', 6),
 ('fwd', 6),
 ('檢驗', 6),
 ('申請', 6),
 ('物流', 6),
 ('正本', 5),
 ('營業稅', 5),
 ('加工', 5),
 ('供應', 5),
 ('書籍', 5),
 ('寄送', 5),
 ('銷售', 5),
 ('付款', 5),
 ('進出口', 5),
 ('ecfa', 5),
 ('fob', 5),
 ('海關', 5),
 ('費用', 5),
 ('運費', 4),
 ('發票', 4),
 ('報價', 4),
 ('金額', 4),
 ('合約', 4),
 ('cif', 4),
 ('代理商', 4),
 ('上海', 4),
 ('方式', 4),
 ('條件', 4),
 ('貨代', 4),
 ('機器', 4),
 ('面試', 4),
 ('請教', 4),
 ('原料', 4),
 ('預審', 4),
 ('尋找', 4),
 ('代理', 4),
 ('相關', 4),
 ('錯誤', 4),
 ('關稅', 4),
 ('歐洲', 4),
 ('集貨', 4),
 ('申報', 4),
 ('出